# Inverse problem of the reconstruction of the Neumann BCs for Laplace's equation

__Formulation of Inverse Problem__

__Direct Problem__
- Equation: $A * x = f$
- $A$: Matrix of Direct Problem
  - shape: ($\text{dof}$, $\text{dof}$)
- $x$: Solution of Direct Problem
  - shape: ($\text{dof}$)
- $f$: Load Vector of Direct Problem
  - shape: ($\text{dof}$)
- $F$: Projection Matrix from Normal Derivative in given points (Mass Matrix over Boundary Elements)
  - shape: ($\text{dof}$, $g$)

__Inverse Problem__
- Equation: $M * y = \text{grad}$
- $M$: Resulting Matrix of Inverse Problem
  - shape: ($3p$, $\text{dof}$)
- $y$: Normal Derivative in Boundary Points
- $\text{grad}$: Concatenation of Grad Vectors ($\text{grad}_x, \text{grad}_y, \text{grad}_z$)

__Additional Variables__
- $P_x, P_y, P_z$: Projection Matrices from Solution to Partial Derivative
  - shape: ($p$, $\text{dof}$)
- $M_x = P_x \times A^{-1} \times F$: Need batching with Direct Solvers (spsolve or better factorized to reduce amount of time)
  - Note: $A$ is singular, so we need to be careful with solving the system
- $M = (M_x, M_y, M_z)$: Concatenation of Inverse Problem Matrices
- $\text{dof}$: Degrees of Freedom of Direct Problem
- $p$: Number of Measuring Points
- $g$: Number of Boundary Points with Known Normal Derivative (Is it all boundary points?)

__About calculating M__

Computing $M$ consists of solving a lot of systems of linear equations:

$$A^{-1} * F = X$$

is same with solving SLAE with matrix rhs:

$$AX = F$$

Result of this multipication can be a big dense (90% density) matrix so we should solve SLAE with one or several columns form $F$.

Probable algorithm for getting $M$ (Under investigation, because it gives big error (e.g. `1e-2` instead of `1e-11`))

```python
for f in F:

  x = spsolve(A, f) # f.shape: (dof, 1)
  
  Mx[i] = P_x @ x
```

__Questions__

1. How to deal with $A$ is singular? It affects calculation of $M$.

2. How to assemble $F$, if we have rhs (source) in direct problem? (If equation is not Laplace's)

## Preprocessing

Import the required libraries

In [1]:
import numpy as np
import scipy as sp
import cupy as cp
import cupy.sparse.linalg as cpsl

from tqdm import tqdm

import pyquasar as pq

np.set_printoptions(precision=3, suppress=False)

Load the `.geo` file and generate the mesh for the problem using GMSH.

`refine_k` is the number of times the mesh is refined.

In [41]:
mesh = pq.Mesh.load("circle_splitted.msh", use_quadratic=True, refine_k=0)
mesh

<Mesh object summary 
	Numeration: global
	Domains: [<MeshDomain object summary
	Material: air
	Total elements number: 10752
	Element type: Tetrahedron 10 NC; Count: 10752
	Boundary type: neumann; Tag: 4; Element type: Triangle 6 NC; Count: 2048.
	Boundary type: neumann; Tag: 5; Element type: Triangle 6 NC; Count: 224.
	Boundary type: neumann; Tag: 6; Element type: Triangle 6 NC; Count: 224.
>]>

In [42]:
mesh.domains[0].vertices[..., 2] = mesh.domains[0].vertices[..., 2] + 1.6

In [43]:
mesh.domains[0].vertices[..., 2].min()

0.0

## Direct Problem

Assemble & solve the direct problem to obtain problem matrix, load vector and the solution `u`.

Note that mesh consists of only Neumann boundary conditions and therefore the solution `u` is need to be orthogonalized.

In [44]:
# Define the materials dictionary
materials = {
  "air": {"air": 0},
}

# Create a list of FemDomain objects from the mesh domains
domains = [pq.FemDomain(domain) for domain in mesh.domains]

# Create a FemProblem object with the domains
problem = pq.FemProblem(domains)

# Assemble the problem using the materials dictionary
problem.assembly(materials)

# Print the degree of freedom count
print(f"DOF: {problem.dof_count}")

problem._matrix = problem._matrix.tocsr()
problem.matrix.data[problem.matrix.indptr[-2] : problem.matrix.indptr[-1]] = 0
problem._matrix = problem._matrix.tocsc()
problem.matrix.data[problem.matrix.indptr[-2] : problem.matrix.indptr[-1]] = 0
problem._matrix[-1, -1] = 1
problem._matrix.eliminate_zeros()

DOF: 16929


## Inverse Problem

Get points of interest

In [45]:
data = np.load("data/data.npz")
xs = data["xs"]
ys = data["ys"]
zs = data["zs"]
grad_x = data["grad_x"]
grad_y = data["grad_y"]
grad_z = data["grad_z"]
data.close()

In [46]:
pts = np.concatenate([xs[:7200, None], ys[:7200, None], zs[:7200, None]], axis=1)
pts.shape

(7200, 3)

Project the solution gradient into the points of interest

In [47]:
proj_grad = problem.project_grad_into(pts, batch_size=128)

100%|██████████| 57/57 [00:02<00:00, 25.76it/s]


In [48]:
grad = np.concatenate([grad_x[:7200], grad_y[:7200], grad_z[:7200]])

Get the Mass matrix over the Neumann type boundary

In [49]:
# POSSIBLE CHECK: F[:boundary_ids.max(),]^-1 @ load -> best approximation of the flow
F = problem.mass_boundary(["neumann"])

In [50]:
F_cp = cp.sparse.csc_matrix(F)

Initialize the inverse problem matrix

In [51]:
m_shape = pts.shape[0], F.shape[1]
Mx = np.zeros(m_shape)
My = np.zeros(m_shape)
Mz = np.zeros(m_shape)

In [52]:
Mx_cp = cp.array(Mx)
My_cp = cp.array(My)
Mz_cp = cp.array(Mz)

Get factorization of direct problem matrix

In [53]:
sp.sparse.linalg.use_solver(useUmfpack = False)

In [54]:
# direct_problem_solve = sp.sparse.linalg.factorized(problem.matrix)

In [55]:
direct_problem_solve_cp = cp.sparse.linalg.factorized(cp.sparse.csc_matrix(problem.matrix))

Iteratively calculate the inverse problem matrix

In [17]:
# for i, f in tqdm(enumerate(F.T), total=F.shape[1]):
#   sol = direct_problem_solve(f.T.toarray().ravel())

#   Mx[:, i] = (proj_grad[0] @ sol).ravel()
#   My[:, i] = (proj_grad[1] @ sol).ravel()
#   Mz[:, i] = (proj_grad[2] @ sol).ravel()

# M = np.concatenate([Mx, My, Mz])

In [56]:
proj_grad_cp = (cp.sparse.csr_matrix(proj_grad[0]), cp.sparse.csr_matrix(proj_grad[1]), cp.sparse.csr_matrix(proj_grad[2]))

In [57]:
for i, f in tqdm(enumerate(F.T), total=F.shape[1]):
  sol = direct_problem_solve_cp(cp.array(f.T.toarray().ravel()))

  Mx_cp[:, i] = (proj_grad_cp[0] @ sol).ravel()
  My_cp[:, i] = (proj_grad_cp[1] @ sol).ravel()
  Mz_cp[:, i] = (proj_grad_cp[2] @ sol).ravel()

M_cp = cp.concatenate([Mx_cp, My_cp, Mz_cp])

100%|██████████| 4994/4994 [01:25<00:00, 58.31it/s]


In [58]:
# M_cp = cp.load("M_refined_circle_splitted.npy")

In [59]:
grad_cp = cp.array(grad)

In [21]:
# M_cp = cp.array(M)

Iteratively calculate the inverse problem solution

In [81]:
res_flow, istop, itn, normr, normar = cpsl.lsmr(M_cp, grad_cp, atol=1e-15, btol=1e-15, maxiter=80000)[:5]

rerr = cp.linalg.norm(M_cp @ res_flow - grad_cp) / np.linalg.norm(grad_cp)

print(f"The reason of stopping: {istop}")
print(f"Number of iterations: {itn}")
print(f"Norm of the residual: {normr:.2e}")
print(f"Norm of modified residual: {normar:.2e}")
print(f"Relative error of inverse problem solution (M @ y = grad): {rerr:.2e}")

The reason of stopping: 7
Number of iterations: 80000
Norm of the residual: 3.54e-01
Norm of modified residual: 5.09e-07
Relative error of inverse problem solution (M @ y = grad): 1.91e-03


In [23]:
# M_pinv_cp = cp.linalg.pinv(M_cp, rcond=1e-11)
# res_flow_cp = M_pinv_cp @ grad_cp

# normr_pinv = cp.linalg.norm(M_cp @ res_flow_cp - grad_cp) / cp.linalg.norm(grad_cp)
# normar_pinv = cp.linalg.norm(M_cp.T @ M_cp @ res_flow_cp - M_cp.T @ grad_cp) / cp.linalg.norm(M_cp.T @ grad_cp)
# print(f"Norm of modified residual: {normar_pinv:.2e}")
# print(f"Norm of residual (M @ y = grad): {normr_pinv:.2e}")

In [82]:
res_flow = cp.asnumpy(res_flow)

Test through direct problem

In [83]:
mat_dict = {
  "air": {"air": 0},
}

test_problem = pq.FemProblem(domains)

# Assemble the problem using the materials dictionary
test_problem.assembly(mat_dict)

# Print the degree of freedom count
print(f"DOF: {test_problem.dof_count}")

test_problem._matrix = test_problem._matrix.tocsr()
test_problem.matrix.data[test_problem.matrix.indptr[-2] : test_problem.matrix.indptr[-1]] = 0
test_problem._matrix = test_problem._matrix.tocsc()
test_problem.matrix.data[test_problem.matrix.indptr[-2] : test_problem.matrix.indptr[-1]] = 0
test_problem._matrix[-1, -1] = 1
test_problem._matrix.eliminate_zeros()

# Get the kernel for Gram-Schmidt orthogonalization
kernel = test_problem.domains[0].kernel

test_problem._load_vector += F @ res_flow

# Solve the problem
test_sol = test_problem.solve(atol=1e-15)

# Perform the Gram-Schmidt orthogonalization
test_sol -= kernel[0] * (test_sol @ kernel[0]) / (kernel[0] @ kernel[0])

DOF: 16929


In [63]:
test_pts = np.concatenate([xs[7200:, None], ys[7200:, None], zs[7200:, None]], axis=1)

In [64]:
test_true_grad = np.concatenate([grad_x[7200:], grad_y[7200:], grad_z[7200:]])

In [65]:
test_proj_grad = test_problem.project_grad_into(test_pts, batch_size=128)

100%|██████████| 1145/1145 [00:44<00:00, 25.76it/s]


In [84]:
test_grad_x = test_proj_grad[0] @ test_sol
test_grad_y = test_proj_grad[1] @ test_sol
test_grad_z = test_proj_grad[2] @ test_sol

test_grad = np.concatenate([test_grad_x, test_grad_y, test_grad_z])

In [85]:
np.linalg.norm(test_grad - test_true_grad) / np.linalg.norm(test_true_grad)

0.5664542467655315

In [86]:
np.abs(test_grad - test_true_grad).mean()

0.08076765971892326